# 과제 : ‘Santander Customer Transaction Prediction’ 데이터 분석
- EDA, feature engineering (변수선택, scaling 등)  
  (생략해도 되고, Kaggle kernel 참고해도 됨)
- PCA를 적용해서 차원을 축소  
  (차원축소의 성능이 안좋으면 원래 데이터 사용가능)
- SVM 학습, 하이퍼파라미터 튜닝


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df=(df.sample(3000).reset_index()).drop('index',axis=1)
#15000,12000,10000,7000 다 해봤는데 7000도 cv한번마다 30분씩 걸려서 제대로 하기 너무 힘들었습니다..
#그래서 작지만 표본 3000으로 해봤습니다

In [4]:
y=df['target']

### null값과 shape 확인

In [5]:
df.shape

(3000, 202)

In [6]:
print("Total number of labels: {}".format(df.shape[0]))
print("Number of 0: {}".format(len(y[y==0])))
print("Number of 1: {}".format(len(y[y==1])))

Total number of labels: 3000
Number of 0: 2699
Number of 1: 301


### data 준비

In [7]:
df.drop('ID_code',axis=1,inplace=True)

In [8]:
#label을 제거
X=df.drop('target',axis=1)

In [ ]:
#train / test split
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics #model evaluation 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Score_check

In [ ]:
# 모델 설정\
classifiers = []
classifiers.append(SVC(kernel='linear'))
classifiers.append(SVC(kernel='rbf'))
classifiers.append(SVC(kernel='poly'))

In [ ]:
def cv_score_check(X_train,y_train):
    cv_results = []
    for classifier in classifiers :
        cv_results.append(cross_val_score(classifier, X_train, y = y_train, scoring='accuracy',cv = 3,n_jobs=-1))
        cv_means = []
    cv_std = []
    for cv_result in cv_results:
        cv_means.append(cv_result.mean())
        cv_std.append(cv_result.std())
    cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":['linear','rbf','poly']})
    cv_res.sort_values(by='CrossValMeans',ascending=False,inplace=True)
    g = sns.barplot("CrossValMeans","Algorithm",data = cv_res, palette="Set3",orient = "h",**{'xerr':cv_std})
    g.set_xlabel("Mean Accuracy")
    g = g.set_title("Cross validation scores")
    display(cv_res)    

In [ ]:
from sklearn.model_selection import cross_val_score
cv_score_check(X_train,y_train) 

# PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
PCA
pca=PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
num_99 = np.argmax(cumsum >= 0.999) + 1
num_9 = np.argmax(cumsum >= 0.9) + 1
num_8 = np.argmax(cumsum >= 0.8) + 1 

In [ ]:
pca8=PCA(n_components=num_8)
pca9=PCA(n_components=num_9)
pca99=PCA(n_components=num_99)

In [ ]:
X_tr_8=pca8.fit_transform(X_train)
X_te_8=pca8.fit_transform(X_test)
X_tr_9=pca9.fit_transform(X_train)
X_te_9=pca9.fit_transform(X_test)
X_tr_99=pca99.fit_transform(X_train)
X_te_99=pca99.fit_transform(X_test)

In [ ]:
svm=SVC(kernel='rbf')

In [ ]:
from sklearn.model_selection import cross_val_score
score=[]
score.append(cross_val_score(svm,X_tr_8,y_train,scoring='accuracy',cv=3,n_jobs=-1))
print('80완료')
score.append(cross_val_score(svm,X_tr_9,y_train,scoring='accuracy',cv=3,n_jobs=-1))
print('90완료')
score.append(cross_val_score(svm,X_tr_99,y_train,scoring='accuracy',cv=3,n_jobs=-1))
print('99완료')

In [ ]:
display(score[0].mean()) # 채택
display(score[1].mean())
display(score[2].mean())

### Hyperparameter search

### Hyperparameter search  -grid search

In [ ]:
from sklearn.svm import SVC
svm_model= SVC()

In [ ]:
tuned_parameters = {
 'C': (np.arange(0.1,1,0.1)) , 'kernel': ['linear'],
 'C': (np.arange(0.1,1,0.1)) , 'gamma': [0.01,0.02,0.03,0.04,0.05], 'kernel': ['rbf']
                   }

In [ ]:
from sklearn.model_selection import GridSearchCV

model_svm = GridSearchCV(svm_model, tuned_parameters,cv=10,scoring='accuracy')

In [ ]:
model_svm.fit(X_tr_8, y_train)
print(model_svm.best_score_)

In [ ]:
print(model_svm.best_params_)

In [ ]:
from sklearn.svm import SVC
svc= SVC(C=0.1, gamma=0.01, kernel='rbf')
svc.fit(X_tr_8,y_train)
y_pred=svc.predict(X_te_8)
print('Accuracy Score:')
print(metrics.accuracy_score(y_test,y_pred))